Το αρχείο train και test dataset τα αρχικά

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

df_train = pd.read_csv('train.csv')
df_train.head()

Using TensorFlow backend.


,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd,PAX
0,2011-12-05,ORD,Chicago,41.978603,-87.904842,EWR,Newark,40.692500,-74.168667,8.352941,5.667243,7
1,2013-03-01,ATL,Atlanta,33.636719,-84.428067,LGA,New York,40.777245,-73.872608,10.421053,10.001754,7
2,2012-02-06,ORD,Chicago,41.978603,-87.904842,BOS,Boston,42.364347,-71.005181,9.250000,7.136821,7
3,2012-12-10,ORD,Chicago,41.978603,-87.904842,SFO,San Francisco,37.618972,-122.374889,8.666667,7.404291,7
4,2012-04-13,SFO,San Francisco,37.618972,-122.374889,JFK,New York,40.639751,-73.778925,14.037037,9.858544,7


In [2]:
df_test = pd.read_csv('test.csv')
df_test.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd
0,2012-10-21,DFW,Dallas-Fort Worth,32.896828,-97.037997,SFO,San Francisco,37.618972,-122.374889,14.600000,11.575837
1,2012-09-13,LAX,Los Angeles,33.942536,-118.408075,ATL,Atlanta,33.636719,-84.428067,14.730769,13.364304
2,2012-09-04,ORD,Chicago,41.978603,-87.904842,IAH,Houston,29.984433,-95.341442,8.470588,5.885551
3,2012-08-13,DEN,Denver,39.861656,-104.673178,PHX,Phoenix,33.434278,-112.011583,8.200000,6.292853
4,2012-09-10,ORD,Chicago,41.978603,-87.904842,SEA,Seattle,47.449000,-122.309306,12.090909,9.138662


ΣΥΝΑΡΤΗΣΗ ΜΕΤΑΤΡΟΠΗΣ ΓΕΩΓΡΑΜΙΚΟΥ ΜΗΚΟΥΣ-ΠΛΑΤΟΥΣ ΑΕΡΟΔΡΟΜΙΟΥ ΑΝΑΧΩΡΗΣΗ-ΑΦΙΞΗΣ ΣΕ ΑΠΟΣΤΑΣΗ ΜΕΤΑΞΥ ΤΩΝ ΔΥΟ ΑΕΡΟΔΡΟΜΙΩΝ

In [3]:
from math import sin, cos, sqrt, atan2, radians

def distance_calculation(lat1,lon1,lat2,lon2): 
    
    R = 6373.0 # approximate radius of earth in km

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

disTr=[None]*len(df_train)
for i in range(len(df_train)):
    disTr[i] = distance_calculation(df_train['LatitudeDeparture'][i],df_train['LongitudeDeparture'][i],df_train['LatitudeArrival'][i],df_train['LongitudeArrival'][i])

disTe=[None]*len(df_test)
for i in range(len(df_test)):
    disTe[i] = distance_calculation(df_test['LatitudeDeparture'][i],df_test['LongitudeDeparture'][i],df_test['LatitudeArrival'][i],df_test['LongitudeArrival'][i])

ΚΑΝΟΝΙΚΟΠΟΙΗΣΗ ΤΗΣ ΜΕΤΑΒΛΗΤΗΣ ΤΗΣ ΑΠΟΣΤΑΣΗΣ ΜΕ ΤΗΝ ΜΕΘΟΔΟ ΤΟΥ STANDARD SCALER(ΠΑΡΕΧΕΤΑΙ ΚΑΙ Ο MINMAXSCALER)

In [4]:
#Normalize distance
#from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
scaler2 = StandardScaler()
#scaler=MinMaxScaler()

#disTrain=disTr.copy()
#disTest=disTe.copy()
disTrain2=disTr.copy()
disTest2=disTe.copy()

#disTrain=np.ravel(disTrain)
#disTest=np.ravel(disTest)
disTrain2=np.ravel(disTrain2)
disTest2=np.ravel(disTest2)

#disTrain=disTrain.reshape(-1, 1)
#disTest=disTest.reshape(-1, 1)
disTrain2=disTrain2.reshape(-1, 1)
disTest2=disTest2.reshape(-1, 1)

#X_scaled = scaler.fit_transform(disTrain)  
#T_scaled = scaler.fit_transform(disTest)
X_scaled2 = scaler2.fit_transform(disTrain2)  
T_scaled2 = scaler2.fit_transform(disTest2)


for i in range(len(df_train)):
    #disTrain[i]=X_scaled[i]
    disTrain2[i]=X_scaled2[i]
    
for i in range(len(df_test)):
    #disTest[i]=T_scaled[i]
    disTest2[i]=T_scaled2[i]

ΔΗΜΙΟΥΡΓΙΑ FEATURE Airports

In [5]:
#Airports train
d=0
a=0
airportTr= [None]*len(df_train)
air=['ATL','ORD','LAX','DFW','DEN','JFK','SFO','CLT','LAS','PHX','IAH','MIA','MCO','EWR','SEA','MSP','DTW','PHL','BOS']
for i in range(len(df_train)):
    a=0
    d=0
    for j in range(len(air)):
        if(df_train['Arrival'][i]==air[j]):
            a=1
        if(df_train['Departure'][i]==air[j]):
            d=1
    if(a==1 and d==1):
        airportTr[i] = 2
            
    elif((a==1 and d!=1) or (a!=1 and d==1)):
        airportTr[i] = 1
    else:
        airportTr[i] = 0
df_train.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd,PAX
0,2011-12-05,ORD,Chicago,41.978603,-87.904842,EWR,Newark,40.692500,-74.168667,8.352941,5.667243,7
1,2013-03-01,ATL,Atlanta,33.636719,-84.428067,LGA,New York,40.777245,-73.872608,10.421053,10.001754,7
2,2012-02-06,ORD,Chicago,41.978603,-87.904842,BOS,Boston,42.364347,-71.005181,9.250000,7.136821,7
3,2012-12-10,ORD,Chicago,41.978603,-87.904842,SFO,San Francisco,37.618972,-122.374889,8.666667,7.404291,7
4,2012-04-13,SFO,San Francisco,37.618972,-122.374889,JFK,New York,40.639751,-73.778925,14.037037,9.858544,7


ΔΗΜΙΟΥΡΓΙΑ FEATURE Airports

In [6]:
#Airports test data
d=0
a=0
airportTe= [None]*len(df_test)
air=['ATL','ORD','LAX','DFW','DEN','JFK','SFO','CLT','LAS','PHX','IAH','MIA','MCO','EWR','SEA','MSP','DTW','PHL','BOS']
for i in range(len(df_test)):
    a=0
    d=0
    for j in range(len(air)):
        if(df_test['Arrival'][i]==air[j]):
            a=1
        if(df_test['Departure'][i]==air[j]):
            d=1
    if(a==1 and d==1):
        airportTe[i] = 2
            
    elif((a==1 and d!=1) or (a!=1 and d==1)):
        airportTe[i] = 1
    else:
        airportTe[i] = 0
df_test.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd
0,2012-10-21,DFW,Dallas-Fort Worth,32.896828,-97.037997,SFO,San Francisco,37.618972,-122.374889,14.600000,11.575837
1,2012-09-13,LAX,Los Angeles,33.942536,-118.408075,ATL,Atlanta,33.636719,-84.428067,14.730769,13.364304
2,2012-09-04,ORD,Chicago,41.978603,-87.904842,IAH,Houston,29.984433,-95.341442,8.470588,5.885551
3,2012-08-13,DEN,Denver,39.861656,-104.673178,PHX,Phoenix,33.434278,-112.011583,8.200000,6.292853
4,2012-09-10,ORD,Chicago,41.978603,-87.904842,SEA,Seattle,47.449000,-122.309306,12.090909,9.138662


ΔΗΜΙΟΥΡΓΙΑ FEATURE IsHolidays

In [7]:
#Date train data
import datetime
from datetime import date
dayTr=[None]*len(df_train)
monthTr = [None]*len(df_train)
yearTr = [None]*len(df_train)
holTr=[None]*len(df_train)
normDate=[None]*len(df_train)
doubleYear=[31,29,31,30,31,30,31,31,30,31,30,31]
oneYear=[31,28,31,30,31,30,31,31,30,31,30,31]
for i in range(len(df_train)):
    ys,ms,ds=df_train['DateOfDeparture'][i].split('-')
    d=int(ds)
    m=int(ms)
    y=int(ys)
    dayTr[i]=d
    monthTr[i]=m
    yearTr[i]=y
    holTr[i]=0
    if(m==12):
        if(d>=18 and d<=31):
            holTr[i]=2
        elif(d>=1 and d<=3):
            holTr[i]=2
    elif(m==1):
         if(d>=1 and d<=6):
            holTr[i]=2
       
    elif(m==9):
         if(d>=1 and d<=7):
            holTr[i]=1
        
    elif(m==8):
        
        if(d>=29 and d<=31):
            holTr[i]=1
        wD=datetime.date(y,m,d).weekday()
        if(wD==4 or wD==5 or wD==6):
            holTr[i]=1 
        
    elif(m==5):
         if(d>=22 and d<=30):
            holTr[i]=1
        
    elif(m==7):
        if(d>=1 and d<=7):
            holTr[i]=1
        wD=datetime.date(y,m,d).weekday()
        if(wD==4 or wD==5 or wD==6):
            holTr[i]=1 
    elif(m==11):
         if(d>=23 and d<=31):
            holTr[i]=2
   

    #NEW ENTRY NORM DATE
    if(y==2012):
        if(m==1):
            normDate[i]=d/366  
        elif(m==2):
             normDate[i]=(doubleYear[0]+d)/366 
        elif(m==3):
             normDate[i]=(doubleYear[0]+doubleYear[1]+d)/366 
        
        elif(m==4):
             normDate[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+d)/366 
        
        elif(m==5):
             normDate[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+d)/366 
        elif(m==6):
             normDate[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+d)/366 
        
        elif(m==7):
            normDate[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+d)/366 
        elif(m==8):
            normDate[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+doubleYear[6]+d)/366 
        elif(m==9):
            normDate[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+doubleYear[6]+doubleYear[7]+d)/366 
        elif(m==10):
            normDate[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+doubleYear[6]+doubleYear[7]+doubleYear[8]+d)/366 
        elif(m==11):
            normDate[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+doubleYear[6]+doubleYear[7]+doubleYear[8]+doubleYear[9]+d)/366 
        elif(m==12):
            normDate[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+doubleYear[6]+doubleYear[7]+doubleYear[8]+doubleYear[9]+doubleYear[10]+d)/366 
    else:
        if(m==1):
            normDate[i]=d/365 
        elif(m==2):
             normDate[i]=(oneYear[0]+d)/365 
        elif(m==3):
             normDate[i]=(oneYear[0]+oneYear[1]+d)/365 
        
        elif(m==4):
             normDate[i]=(oneYear[0]+oneYear[1]+oneYear[2]+d)/365 
        
        elif(m==5):
             normDate[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+d)/365 
        elif(m==6):
             normDate[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+d)/365 
        
        elif(m==7):
            normDate[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+d)/365 
        elif(m==8):
            normDate[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+oneYear[6]+d)/365 
        elif(m==9):
            normDate[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+oneYear[6]+oneYear[7]+d)/365 
        elif(m==10):
            normDate[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+oneYear[6]+oneYear[7]+oneYear[8]+d)/365 
        elif(m==11):
            normDate[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+oneYear[6]+oneYear[7]+oneYear[8]+oneYear[9]+d)/365 
        elif(m==12):
            normDate[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+oneYear[6]+oneYear[7]+oneYear[8]+oneYear[9]+oneYear[10]+d)/365 

In [8]:
#Date Test data
import datetime
dayTe=[None]*len(df_test)
monthTe = [None]*len(df_test)
yearTe = [None]*len(df_test)
holTe=[None]*len(df_test)
normDate1=[None]*len(df_test)
doubleYear=[31,29,31,30,31,30,31,31,30,31,30,31]
oneYear=[31,28,31,30,31,30,31,31,30,31,30,31]
for i in range(len(df_test)):
    ys,ms,ds=df_test['DateOfDeparture'][i].split('-')
    d=int(ds)
    m=int(ms)
    y=int(ys)
    dayTe[i]=d
    monthTe[i]=m
    yearTe[i]=y
    holTe[i]=0
    if(m==12):
        if(d>=18 and d<=31):
            holTe[i]=2
        elif(d>=1 and d<=3):
            holTe[i]=2
        
    elif(m==1):
         if(d>=1 and d<=6):
            holTe[i]=2
        
    elif(m==9):
         if(d>=1 and d<=7):
            holTe[i]=1
        
    elif(m==8):
        if(d>=29 and d<=31):
            holTe[i]=1
        wD=datetime.date(y,m,d).weekday()
        if(wD==4 or wD==5 or wD==6):
            holTe[i]=1 
    elif(m==5):
         if(d>=22 and d<=30):
            holTe[i]=1
        
    elif(m==7):
        if(d>=1 and d<=7):
            holTe[i]=1
        wD=datetime.date(y,m,d).weekday()
        if(wD==4 or wD==5 or wD==6):
            holTe[i]=1 
    elif(m==11):
         if(d>=23 and d<=31):
            holTe[i]=2

            
    #NEW ENTRY NORM DATE
    if(y==2012):
        if(m==1):
            normDate1[i]=d/366  
        elif(m==2):
             normDate1[i]=(doubleYear[0]+d)/366 
        elif(m==3):
             normDate1[i]=(doubleYear[0]+doubleYear[1]+d)/366 
        
        elif(m==4):
             normDate1[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+d)/366 
        
        elif(m==5):
             normDate1[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+d)/366 
        elif(m==6):
             normDate1[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+d)/366 
        
        elif(m==7):
            normDate1[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+d)/366 
        elif(m==8):
            normDate1[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+doubleYear[6]+d)/366 
        elif(m==9):
            normDate1[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+doubleYear[6]+doubleYear[7]+d)/366 
        elif(m==10):
            normDate1[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+doubleYear[6]+doubleYear[7]+doubleYear[8]+d)/366 
        elif(m==11):
            normDate1[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+doubleYear[6]+doubleYear[7]+doubleYear[8]+doubleYear[9]+d)/366 
        elif(m==12):
            normDate1[i]=(doubleYear[0]+doubleYear[1]+doubleYear[2]+doubleYear[3]+doubleYear[4]+doubleYear[5]+doubleYear[6]+doubleYear[7]+doubleYear[8]+doubleYear[9]+doubleYear[10]+d)/366 
    else:
        if(m==1):
            normDate1[i]=d/365 
        elif(m==2):
             normDate1[i]=(oneYear[0]+d)/365 
        elif(m==3):
             normDate1[i]=(oneYear[0]+oneYear[1]+d)/365 
        
        elif(m==4):
             normDate1[i]=(oneYear[0]+oneYear[1]+oneYear[2]+d)/365 
        
        elif(m==5):
             normDate1[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+d)/365 
        elif(m==6):
             normDate1[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+d)/365 
        
        elif(m==7):
            normDate1[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+d)/365 
        elif(m==8):
            normDate1[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+oneYear[6]+d)/365 
        elif(m==9):
            normDate1[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+oneYear[6]+oneYear[7]+d)/365 
        elif(m==10):
            normDate1[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+oneYear[6]+oneYear[7]+oneYear[8]+d)/365 
        elif(m==11):
            normDate1[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+oneYear[6]+oneYear[7]+oneYear[8]+oneYear[9]+d)/365 
        elif(m==12):
            normDate1[i]=(oneYear[0]+oneYear[1]+oneYear[2]+oneYear[3]+oneYear[4]+oneYear[5]+oneYear[6]+oneYear[7]+oneYear[8]+oneYear[9]+oneYear[10]+d)/365 

ΚΑΝΟΝΙΚΟΠΟΙΗΣΗ ΤΩΝ ΠΟΣΟΤΙΚΩΝ ΜΕΤΑΒΛΗΤΩΝ ΜΕ ΤΟΝ STANDARDSCALER

In [9]:
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
df_train[['LongitudeDeparture','LatitudeDeparture','LongitudeArrival','LatitudeArrival','WeeksToDeparture','std_wtd']]=scaler1.fit_transform(df_train[['LongitudeDeparture','LatitudeDeparture','LongitudeArrival','LatitudeArrival','WeeksToDeparture','std_wtd']])
df_train.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd,PAX
0,2011-12-05,ORD,Chicago,0.893873,0.334627,EWR,Newark,0.628311,1.114971,-1.110302,-1.382558,7
1,2013-03-01,ATL,Atlanta,-0.894398,0.533909,LGA,New York,0.646326,1.131891,-0.371087,0.643282,7
2,2012-02-06,ORD,Chicago,0.893873,0.334627,BOS,Boston,0.983713,1.295768,-0.789662,-0.695715,7
3,2012-12-10,ORD,Chicago,0.893873,0.334627,SFO,San Francisco,-0.025061,-1.640075,-0.998166,-0.570706,7
4,2012-04-13,SFO,San Francisco,-0.040712,-1.641137,JFK,New York,0.617098,1.137245,0.921391,0.576349,7


In [10]:
scaler1 = StandardScaler()
df_test[['LongitudeDeparture','LatitudeDeparture','LongitudeArrival','LatitudeArrival','WeeksToDeparture','std_wtd']]=scaler1.fit_transform(df_test[['LongitudeDeparture','LatitudeDeparture','LongitudeArrival','LatitudeArrival','WeeksToDeparture','std_wtd']])
df_test.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd
0,2012-10-21,DFW,Dallas-Fort Worth,-1.046445,-0.185228,SFO,San Francisco,-0.043391,-1.665838,1.091168,1.337355
1,2012-09-13,LAX,Los Angeles,-0.820550,-1.411666,ATL,Atlanta,-0.889163,0.494646,1.137155,2.161748
2,2012-09-04,ORD,Chicago,0.915406,0.338927,IAH,Houston,-1.664855,-0.126702,-1.064346,-1.285581
3,2012-08-13,DEN,Denver,0.458101,-0.623415,PHX,Phoenix,-0.932159,-1.075808,-1.159503,-1.097835
4,2012-09-10,ORD,Chicago,0.915406,0.338927,SEA,Seattle,2.044363,-1.662104,0.208802,0.213940


ΤΑ 2 ΕΠΟΜΕΝΑ ΚΕΛΙΑ ΜΠΑΙΝΟΥΝ ΜΕ ΤΗΝ ΠΡΟΥΠΟΘΕΣΗ ΟΤΙ ΘΕΛΟΥΜΕ ΝΑ ΚΑΝΟΥΜΕ MINMAXSCALER

In [11]:
#from sklearn.preprocessing import MinMaxScaler
#import numpy as np
#scaler = MinMaxScaler()
#df_train[['LongitudeDeparture','LatitudeDeparture','LongitudeArrival','LatitudeArrival','WeeksToDeparture','std_wtd']]=scaler.fit_transform(df_train[['LongitudeDeparture','LatitudeDeparture','LongitudeArrival','LatitudeArrival','WeeksToDeparture','std_wtd']])
#df_train.head()

In [12]:
#scaler = MinMaxScaler()
#df_test[['LongitudeDeparture','LatitudeDeparture','LongitudeArrival','LatitudeArrival','WeeksToDeparture','std_wtd']]=scaler.fit_transform(df_test[['LongitudeDeparture','LatitudeDeparture','LongitudeArrival','LatitudeArrival','WeeksToDeparture','std_wtd']])
#df_test.head()

ΓΡΑΦΟΥΜΕ ΞΑΝΑ ΤΟ TEST KAI TRAIN DATASET ΣΕ 2 ΝΕΑ ΑΡΧΕΙΑ ΩΣΤΕ ΝΑ ΤΑ ΔΙΑΒΑΣΟΥΜΕ ΑΠΟ ΚΕΙ(ΤΑ ΓΡΑΦΟΥΜΕ ΓΙΑ ΤΗΝ ΠΕΡΙΠΤΩΣΗ ΠΟΥ ΑΠΛΑ ΘΕΛΟΥΜΕ ΝΑ ΑΝΟΙΞΟΥΜΕ ΤΑ ΝΕΑ DATASETS ΣΕ ΕΝΑ EXCEL ΚΑΙ ΝΑ ΔΟΥΜΕ ΤΑ ΔΕΔΟΜΕΝΑ)

In [13]:
import csv
with open('new_train.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['DateOfDeparture', 'Departure','CityDeparture','LongitudeDeparture','LatitudeDeparture','Arrival','CityArrival','LongitudeArrival','LatitudeArrival','WeeksToDeparture','std_wtd','Distance','Airports','IsHolidays','PAX'])
    for i in range(len(df_train)):
        writer.writerow([df_train['DateOfDeparture'][i],df_train['Departure'][i],df_train['CityDeparture'][i],df_train['LongitudeDeparture'][i],df_train['LatitudeDeparture'][i],df_train['Arrival'][i],df_train['CityArrival'][i],df_train['LongitudeArrival'][i],df_train['LatitudeArrival'][i],df_train['WeeksToDeparture'][i],df_train['std_wtd'][i],disTrain2.T[0][i],airportTr[i],holTr[i],df_train['PAX'][i]])

In [14]:
import csv
with open('new_test.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['DateOfDeparture', 'Departure','CityDeparture','LongitudeDeparture','LatitudeDeparture','Arrival','CityArrival','LongitudeArrival','LatitudeArrival','WeeksToDeparture','std_wtd','Distance','Airports','IsHolidays'])
    for i in range(len(df_test)):
        writer.writerow([df_test['DateOfDeparture'][i],df_test['Departure'][i],df_test['CityDeparture'][i],df_test['LongitudeDeparture'][i],df_test['LatitudeDeparture'][i],df_test['Arrival'][i],df_test['CityArrival'][i],df_test['LongitudeArrival'][i],df_test['LatitudeArrival'][i],df_test['WeeksToDeparture'][i],df_test['std_wtd'][i],disTest2.T[0][i],airportTe[i],holTe[i]])

ΦΟΡΤΩΝΟΥΜΕ ΤΑ ΝΕΑ ΤΡΟΠΟΠΟΙΗΜΕΝΑ DATASETS

In [15]:
df_train = pd.read_csv('new_train.csv')
df_train.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd,Distance,Airports,IsHolidays,PAX
0,2011-12-05,ORD,Chicago,0.893873,0.334627,EWR,Newark,0.628311,1.114971,-1.110302,-1.382558,-0.239266,2,0,7
1,2013-03-01,ATL,Atlanta,-0.894398,0.533909,LGA,New York,0.646326,1.131891,-0.371087,0.643282,-0.466538,1,0,7
2,2012-02-06,ORD,Chicago,0.893873,0.334627,BOS,Boston,0.983713,1.295768,-0.789662,-0.695715,-0.008578,2,0,7
3,2012-12-10,ORD,Chicago,0.893873,0.334627,SFO,San Francisco,-0.025061,-1.640075,-0.998166,-0.570706,1.272492,2,0,7
4,2012-04-13,SFO,San Francisco,-0.040712,-1.641137,JFK,New York,0.617098,1.137245,0.921391,0.576349,2.302023,2,0,7


In [16]:
df_test = pd.read_csv('new_test.csv')
df_test.head()

,DateOfDeparture,Departure,CityDeparture,LongitudeDeparture,LatitudeDeparture,Arrival,CityArrival,LongitudeArrival,LatitudeArrival,WeeksToDeparture,std_wtd,Distance,Airports,IsHolidays
0,2012-10-21,DFW,Dallas-Fort Worth,-1.046445,-0.185228,SFO,San Francisco,-0.043391,-1.665838,1.091168,1.337355,0.590574,2,0
1,2012-09-13,LAX,Los Angeles,-0.820550,-1.411666,ATL,Atlanta,-0.889163,0.494646,1.137155,2.161748,1.215239,2,0
2,2012-09-04,ORD,Chicago,0.915406,0.338927,IAH,Houston,-1.664855,-0.126702,-1.064346,-1.285581,-0.712174,2,1
3,2012-08-13,DEN,Denver,0.458101,-0.623415,PHX,Phoenix,-0.932159,-1.075808,-1.159503,-1.097835,-0.697832,2,0
4,2012-09-10,ORD,Chicago,0.915406,0.338927,SEA,Seattle,2.044363,-1.662104,0.208802,0.213940,1.245372,2,0


In [17]:
seed = 7
np.random.seed(seed)

ΠΕΤΑΜΕ ΤΙΣ ΣΤΥΛΕΣ CityDeparture LongitudeDeparture LatitudeDeparture CityArrival LongitudeArrival LatitudeArrival ΚΑΙ ΚΡΑΤΑΜΕ ΤΙΣ ΥΠΟΛΟΙΠΕΣ. ΑΠΟ ΤΟ TRAIN DATASET ΠΕΤΑΜΕ ΚΑΙ ΤΗΝ ΣΤΥΛΗ PAX ΜΕ ΤΑ LABELS ΑΦΟΥ ΤΗΝ ΑΠΟΘΗΚΕΥΣΟΥΜΕ ΠΡΩΤΑ ΣΕ ΕΝΑ ΝΕΟ ΠΙΝΑΚΑ ΜΟΝΗ ΤΗΣ

In [18]:
y_train = df_train[['PAX']]

from sklearn.model_selection import train_test_split
#df_train, df_test, y_train, y_test = train_test_split(df_train, y_train, test_size=0.4, random_state=42)

df_train.drop(df_train.columns[[2,3,4,6,7,8,9,14]], axis=1, inplace=True)
df_train.head()

,DateOfDeparture,Departure,Arrival,std_wtd,Distance,Airports,IsHolidays
0,2011-12-05,ORD,EWR,-1.382558,-0.239266,2,0
1,2013-03-01,ATL,LGA,0.643282,-0.466538,1,0
2,2012-02-06,ORD,BOS,-0.695715,-0.008578,2,0
3,2012-12-10,ORD,SFO,-0.570706,1.272492,2,0
4,2012-04-13,SFO,JFK,0.576349,2.302023,2,0


In [19]:
df_test.drop(df_test.columns[[2,3,4,6,7,8,9]], axis=1, inplace=True)
df_test.head()

,DateOfDeparture,Departure,Arrival,std_wtd,Distance,Airports,IsHolidays
0,2012-10-21,DFW,SFO,1.337355,0.590574,2,0
1,2012-09-13,LAX,ATL,2.161748,1.215239,2,0
2,2012-09-04,ORD,IAH,-1.285581,-0.712174,2,1
3,2012-08-13,DEN,PHX,-1.097835,-0.697832,2,0
4,2012-09-10,ORD,SEA,0.213940,1.245372,2,0


In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df_train['Departure'])
df_train['Departure'] = le.transform(df_train['Departure'])
df_train['Arrival'] = le.transform(df_train['Arrival'])
df_test['Departure'] = le.transform(df_test['Departure'])
df_test['Arrival'] = le.transform(df_test['Arrival'])
le = LabelEncoder()
le.fit(df_train['DateOfDeparture'])
df_train['DateOfDeparture'] = le.transform(df_train['DateOfDeparture'])
df_test['DateOfDeparture'] = le.transform(df_test['DateOfDeparture'])

In [21]:
from sklearn.preprocessing import OneHotEncoder
cf = [0,1,2,5,6]
enc1 = OneHotEncoder(categorical_features=cf,sparse=False)#categorical_features=cf,sparse=False
enc1.fit(df_train)
X_train = enc1.transform(df_train)
X_test = enc1.transform(df_test)
X_train.shape

C:\Users\IAKOVOS\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\IAKOVOS\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


(8899, 599)

In [22]:
#ONE HOT Encode output variables
import numpy as np
enc2 = OneHotEncoder(sparse=False)
enc2.fit(y_train)
Y_train = enc2.transform(y_train)
#Y_test = enc2.transform(y_test)
Y_train.shape

C:\Users\IAKOVOS\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(8899, 8)

ΣΤΗΝ ΠΕΡΙΠΤΩΣΗ ΠΟΥ ΘΕΛΟΥΜΕ ΝΑ ΚΑΝΟΥΜΕ CROSS-VALIDATION

In [23]:
#X_test, X_val,Y_test,Y_val = train_test_split(X_test, Y_test, test_size=0.2, random_state=42)

In [24]:
import keras
from keras.models import Sequential
model = Sequential()

In [25]:
from keras.layers import Dense
from keras.layers import Dropout
d=X_train.shape[1]
#input layer->d neurones | 1 hidden layer 300 neurones| 1 hidden layer 150 neurones|output layer 8 neurones
model.add(Dense(300, input_dim=d, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

In [26]:
# Compile model
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.SGD(lr=0.007,momentum=0.9))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300)               180000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 150)               45150     
_________________________________________________________________
dropout_2 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 1208      
Total params: 226,358
Trainable params: 226,358
Non-trainable params: 0
_________________________________________________________________


ΣΤΗΝ ΠΕΡΙΠΤΩΣΗ ΠΟΥ ΘΕΛΟΥΜΕ ΝΑ ΚΑΝΟΥΜΕ CROSS-VALIDATION KAI NA ΒΡΟΥΜΕ ΤΙΣ ΚΑΤΑΛΛΗΛΕΣ ΠΑΡΑΜΕΤΡΟΥΣ ΓΙΑ ΤΟ ΜΟΝΤΕΛΟ ΜΑΣ

In [28]:
#from keras.layers.core import Activation
#from keras.callbacks import EarlyStopping
#from keras.callbacks import ModelCheckpoint
#from keras import metrics
#monitor=EarlyStopping(monitor='val_loss',min_delta=1e-3,patience=20,verbose=1,mode='auto')
#checkpointer= ModelCheckpoint(filepath="best_weights.hdf5",verbose=0,save_best_only=True)#save best model
#model.fit(X_train, Y_train, validation_data=(X_val,Y_val),callbacks=[monitor,checkpointer], epochs=200, batch_size=200, verbose=1)
#model.load_weights('best_weights.hdf5')#load weights from best model

In [29]:

#score=model.evaluate(X_test,Y_test,batch_size=10)
#model.metrics_names
#print(score)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):
    model.fit(X_train[train], Y_train[train], epochs=150, batch_size=10, verbose=0)
    # evaluate the model
    scores = model.evaluate(X_test[test], Y_test[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

In [32]:
model.fit(X_train, Y_train, epochs=200, batch_size=200, verbose=1) #validation_split=0.2

Train on 7119 samples, validate on 1780 samples
Epoch 1/200
7119/7119 [==============================] - ETA: 0s - loss: 2.035 - 1s 150us/step - loss: 2.0266 - val_loss: 1.9219
Epoch 2/200
7119/7119 [==============================] - 1s 77us/step - loss: 1.9112 - val_loss: 1.8599
Epoch 3/200
7119/7119 [==============================] - 1s 73us/step - loss: 1.8770 - val_loss: 1.8406
Epoch 4/200
7119/7119 [==============================] - 1s 80us/step - loss: 1.8661 - val_loss: 1.8285
Epoch 5/200
7119/7119 [==============================] - 1s 82us/step - loss: 1.8482 - val_loss: 1.8176
Epoch 6/200
7119/7119 [==============================] - 1s 78us/step - loss: 1.8406 - val_loss: 1.8078
Epoch 7/200
7119/7119 [==============================] - 1s 84us/step - loss: 1.8265 - val_loss: 1.7974
Epoch 8/200
7119/7119 [==============================] - 1s 89us/step - loss: 1.8178 - val_loss: 1.7872
Epoch 9/200
7119/7119 [==============================] - 1s 92us/step - loss: 1.8102 - val_loss

7119/7119 [==============================] - 1s 84us/step - loss: 0.8761 - val_loss: 1.0342
Epoch 156/200
7119/7119 [==============================] - 1s 96us/step - loss: 0.8833 - val_loss: 1.0341
Epoch 157/200
7119/7119 [==============================] - 1s 85us/step - loss: 0.8757 - val_loss: 1.0302
Epoch 158/200
7119/7119 [==============================] - 1s 84us/step - loss: 0.8642 - val_loss: 1.0307
Epoch 159/200
7119/7119 [==============================] - 1s 72us/step - loss: 0.8584 - val_loss: 1.0338
Epoch 160/200
7119/7119 [==============================] - 1s 78us/step - loss: 0.8605 - val_loss: 1.0324
Epoch 161/200
7119/7119 [==============================] - 1s 84us/step - loss: 0.8510 - val_loss: 1.0247
Epoch 162/200
7119/7119 [==============================] - 1s 82us/step - loss: 0.8474 - val_loss: 1.0324
Epoch 163/200
7119/7119 [==============================] - 1s 82us/step - loss: 0.8434 - val_loss: 1.0264
Epoch 164/200
7119/7119 [==============================] - 1

ΠΡΟΒΛΕΨΗ ΓΙΑ ΤΟ TEST SET 

In [33]:
from sklearn.metrics import f1_score
y_pred = model.predict(X_test, batch_size=10)

for i in range(len(y_pred)):
    mx=-1
    k=0
    for j in range(8):
        if(mx<y_pred[i][j]):
            mx=y_pred[i][j]
            k=j
    for j in range(8):
        if(k==j):
            y_pred[i][j]=1.0
        else:
            y_pred[i][j]=0.0

y_pred=enc2.inverse_transform(y_pred)
#y_test =enc2.inverse_transform(Y_test)
#f1_score(y_test, y_pred, average='micro')

ΕΓΡΑΦΗ ΑΡΧΕΙΟΥ ΜΕ ΤΙΣ ΠΡΟΒΛΕΨΕΙΣ ΤΟΥ ΜΟΝΤΕΛΟΥ ΜΑΣ

In [34]:
import csv
with open('y_predMLP.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Id', 'Label'])
    for i in range(y_pred.shape[0]):
        writer.writerow([i, int(y_pred[i][0])])

In [ ]:
#from sklearn.metrics import accuracy_score
#print("Accuracy:", 100*accuracy_score(y_test, y_pred))